- 전처리를 안했을 때, 성능이 더 좋더라. 
- 모든 모델을 그렇게 수정해보자.


## Library Import

In [20]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [21]:
# !pip install transformers

In [22]:
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
# import hanja
# from imblearn.over_sampling import RandomOverSampler
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from transformers import AdamW
from collections import defaultdict
import random
import os

import gc
from tqdm import tqdm

import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from sklearn.metrics import f1_score

## Data & Preprocessing & Seed

In [23]:
# def preprocessing(title):
#     x = title[:]
#     x = re.sub('><', ' ', x)
#     x = re.sub('[ㄱ-ㅎ!@#$%^&*(),./?1-9]', ' ', x)
#     return x.strip()

In [24]:
# Train data
origin_train = pd.read_csv('data/train_ml2.csv')
origin_train = origin_train.dropna()
origin_train = origin_train.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_train.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#origin_train['sentence'] = origin_train['sentence'].apply(preprocessing)

# Test data
origin_test = pd.read_csv("data/test_ml2.csv")
origin_test = origin_test.loc[:,['sentence', '범죄', '혐오', '선정', '비도덕 아님', '폭력', '비난', '욕설', '차별']]
origin_test.columns = ['sentence', '범죄', '혐오', '선정', '비도덕아님', '폭력', '비난', '욕설', '차별']
#test['sentence'] = test['sentence'].apply(preprocessing)


In [25]:
# 외부데이터

df = pd.read_table('data/한국어욕설데이터.txt', sep='|', names=['sentence','lbl'])
df = df[df['lbl'] == 0]
df = df.dropna()
df = df[df['sentence'] != ""]

df['범죄'] = 0
df['혐오'] = 0
df['선정'] = 0
df['폭력'] = 0
df['비난'] = 0
df['욕설'] = 0
df['차별'] = 0
df['lbl'] = 1
df.columns = ['sentence', '비도덕아님','범죄','혐오','선정','폭력','비난','욕설','차별']

In [26]:
origin_train = pd.concat([origin_train, df])# df는 한국어욕설데이터
origin_train['sentence'] = origin_train['sentence']#.apply(preprocessing)
origin_train = origin_train[origin_train['sentence'] != ""] # 전처리후 공백이 되면 제거
print("Train Data 길이 :",len(origin_train)) #13,720 -> 이 코드 반복시 Train Data 길이 증가할 수 있음
print("Test Data 길이 :",len(origin_test))



Train Data 길이 : 13785
Test Data 길이 : 1038


In [27]:
import os
def set_seeds(seed=42) :
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
set_seeds()

In [28]:
model_result = pd.DataFrame()

### Electra-kor-base

In [29]:
from transformers import ElectraTokenizerFast, ElectraModel, AdamW, ElectraForSequenceClassification

PRE_TRAINED_MODEL_NAME = "kykim/electra-kor-base"
tokenizer = ElectraTokenizerFast.from_pretrained(PRE_TRAINED_MODEL_NAME)
device = torch.device("cuda")

tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/albert-kor-base")

# electra-kor-base
# tokenizer : kykim/bert-kor-base


class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame) :   
    self.dataset = DataFrame
    self.tokenizer = tokenizer

    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [30]:

# 7가지 Label을 각각 예측하는 모델을 만든다.
label = ['범죄', '혐오', '선정', '폭력', '비난', '욕설', '차별']

for lbl in label : 
    
    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    # 위의 모델과 다른 부분
    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)
    gc.collect()
    torch.cuda.empty_cache()
    EPOCHS = 10
    batch_size = 128
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    total_steps = len(train_data_loader) * EPOCHS

    # 모델 학습 파라미터
    warmup_ratio = 0.1
    NUM_OF_MODELS = 1
    import random
    losses = []
    accuracy = []

    model = ElectraForSequenceClassification.from_pretrained("kykim/electra-kor-base", num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=total_steps)

    for i in range(EPOCHS):
        total_loss = 0.0
        correct = 0
        total = 0
        batches = 0

        model.train()

        for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_data_loader):
            optimizer.zero_grad()
            y_batch = y_batch.to(device)
            y_batch = y_batch.long()
            y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
            loss = F.cross_entropy(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

            _, predicted = torch.max(y_pred, 1)
            correct += (predicted == y_batch).sum()
            total += len(y_batch)

            batches += 1
            if batches % 100 == 0:
                print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
            
        losses.append(total_loss)
        accuracy.append(correct.float() / total)
        print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    # torch.save(model, './Electra_kor_base.model')
    model.eval()

    for input_ids_batch, attention_masks_batch in tqdm(test_data_loader):
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        _, predicted = torch.max(y_pred, 1)
        answer.append(predicted.item())
            
    globals()['{}'.format(lbl)] = answer
    gc.collect()
    torch.cuda.empty_cache()


Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at kykim/electra-kor-base and are newly initialized: ['classifier.dense.weight', 'classif

Batch Loss:  12.674325246363878 Accuracy:  tensor(0.9686, device='cuda:0')


100%|██████████| 108/108 [00:56<00:00,  1.90it/s]


Train Loss : 13.491449724882841 Accuracy : tensor(0.9694, device='cuda:0')


 93%|█████████▎| 100/108 [00:52<00:04,  1.89it/s]

...

 93%|█████████▎| 100/108 [00:53<00:04,  1.88it/s]

Batch Loss:  8.518245844170451 Accuracy:  tensor(0.9833, device='cuda:0')


100%|██████████| 108/108 [00:57<00:00,  1.88it/s]


Train Loss : 9.018239129334688 Accuracy : tensor(0.9837, device='cuda:0')


100%|██████████| 1038/1038 [00:21<00:00, 49.21it/s]


In [31]:

# kykim/electra-kor-base
model_name = "electra-kor-base"

# Test Data 정답
test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))
  
# 비도덕아님 인 경우 예측 결과 만들기
# 비도덕아님은 다른 케이스가 모두 False 인 경우에 True 이다.
moral = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        moral.append(1)
    else :
        moral.append(0)


# 정답 비교
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print("맞춘 갯수 :", count)

accuracy = count / len(origin_test)
print("Label 예측 accuracy :", accuracy)      

# 결과값 저장
result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : moral, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}

from sklearn.metrics import f1_score
pred = []
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    pred.append(data)
f1 = f1_score(test_label, pred, average='micro')


result = pd.DataFrame(result)
result.to_csv("result/final_v3_{}.csv".format(model_name))

test_result = pd.DataFrame({"model_name" : [model_name],
                                    "accuracy" : [accuracy],
                                    "f1_score" : [f1]})
model_result = model_result.append(test_result, ignore_index=True)


맞춘 갯수 : 741
Label 예측 accuracy : 0.7138728323699421


In [32]:
model_result # No 전처리, batch 128으로만 변경한 경우임

,model_name,accuracy,f1_score
0,electra-kor-base,0.713873,0.775798


### Kobert

In [33]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [34]:

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)
    
    

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [35]:

device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

label = ['범죄', '혐오', '선정', '폭력', '비난', '욕설', '차별']

# label이 8개 이렇게 되어있고

max_len = 64 # 64로 바꾸어서 확인도해보기
batch_size = 128
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 50
learning_rate =  1e-5


for lbl in label : 
        
    
    train = origin_train.copy()
    test = origin_test.copy()
    
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})
    
    dataset = []
    for title, label in zip(train.title.to_list(), train.topic_idx.to_list()):
      dataset.append([title,label])
    dataset_train = dataset[:].copy()
    dataset_test = random.sample(dataset,2000).copy() # random 으로 2000개 뽑게하기

    len(dataset_train), len(dataset_test)

    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

## Setting parameters
    data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
    data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

    train_dataloader = torch.utils.data.DataLoader(data_train,  shuffle=True,batch_size=batch_size, num_workers=4)
    test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

    gc.collect()
    torch.cuda.empty_cache()

    # 결과 확인
    valid_dataset = []
    for title, label in zip(df_valid.title.to_list(), np.zeros(len(df_valid))):
      valid_dataset.append([title,label])
    real_test_data = BERTDataset(valid_dataset, 0, 1, tok, max_len, True, False)
    real_test_dataloader = torch.utils.data.DataLoader(real_test_data, batch_size=1, num_workers=4)



    NUM_OF_MODELS = 1
    import random
    for NUM in range(NUM_OF_MODELS):
      dr_r = 0.5
      print("epoch: ",NUM+1)
      print("drop out rate: ",dr_r)
      model = BERTClassifier(bertmodel,  dr_rate=dr_r ).to(device)
      # Prepare optimizer and schedule (linear warmup and decay)
      no_decay = ['bias', 'LayerNorm.weight']
      optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
      ]
      optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
      loss_fn = nn.CrossEntropyLoss()
      t_total = len(train_dataloader) * num_epochs
      warmup_step = int(t_total * warmup_ratio)
      scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
      for e in range(num_epochs):
          train_acc = 0.0
          test_acc = 0.0
          model.train()
          for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
              optimizer.zero_grad()
              token_ids = token_ids.long().to(device)
              segment_ids = segment_ids.long().to(device)
              valid_length= valid_length
              label = label.long().to(device)
              out = model(token_ids, valid_length, segment_ids)
              loss = loss_fn(out, label)
              loss.backward()
              torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
              optimizer.step()
              scheduler.step()  # Update learning rate schedule
              train_acc += calc_accuracy(out, label)
              if batch_id % log_interval == 0:
                  print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
          print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
          model.eval()
          for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
              token_ids = token_ids.long().to(device)
              segment_ids = segment_ids.long().to(device)
              valid_length= valid_length
              label = label.long().to(device)
              out = model(token_ids, valid_length, segment_ids)
              test_acc += calc_accuracy(out, label)
          print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
      answer=[]
      model.eval()
      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(real_test_dataloader)):
              token_ids = token_ids.long().to(device)
              segment_ids = segment_ids.long().to(device)
              valid_length= valid_length
              label = label.long().to(device)
              out = model(token_ids, valid_length, segment_ids)
              max_vals, max_indices = torch.max(out, 1)
              answer.append(max_indices[0].item())
      globals()['{}'.format(lbl)] = answer
      gc.collect()
      torch.cuda.empty_cache()

using cached model
using cached model
using cached model
epoch:  1
drop out rate:  0.5


/tmp/ipykernel_10861/1340642537.py:83: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/108 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_10861/1340642537.py:100: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/16 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/1038 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [36]:

model_name = "kobert"

# Test Data 정답
test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))
  
# 비도덕아님 인 경우 예측 결과 만들기
# 비도덕아님은 다른 케이스가 모두 False 인 경우에 True 이다.
moral = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        moral.append(1)
    else :
        moral.append(0)


# 정답 비교
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print("맞춘 갯수 :", count)

accuracy = count / len(origin_test)
print("Label 예측 accuracy :", accuracy)      

# 결과값 저장
result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : moral, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}


result = pd.DataFrame(result)
result.to_csv("result/final_v3_{}.csv".format(model_name))

from sklearn.metrics import f1_score
pred = []
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    pred.append(list(data))
f1 = f1_score(test_label, pred, average='micro')



test_result = pd.DataFrame({"model_name" : [model_name],
                                    "accuracy" : [accuracy],
                                    "f1_score" : [f1]})
model_result = model_result.append(test_result, ignore_index=True)

맞춘 갯수 : 686
Label 예측 accuracy : 0.6608863198458574


In [37]:
model_result

,model_name,accuracy,f1_score
0,electra-kor-base,0.713873,0.775798
1,kobert,0.660886,0.738848


### Ko-electra

In [38]:
class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [39]:
# koelectra


label = ['범죄', '혐오', '선정', '폭력', '비난', '욕설', '차별']
device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

# 모델 학습 파라미터
epochs = 10
batch_size = 128
warmup_ratio=0.1

for lbl in label : 


    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic':labels_t.reshape(-1)})


    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)


    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t_total = len(train_loader) * epochs
    gc.collect()
    torch.cuda.empty_cache()

    losses = []
    accuracy = []
    model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)

    for i in range(epochs):
      total_loss = 0.0
      correct = 0
      total = 0
      batches = 0

      model.train()

      for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_batch = y_batch.long()
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
          print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
          
      losses.append(total_loss)
      accuracy.append(correct.float() / total)
      print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    model.eval()
    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      answer.append(predicted.item())

    globals()['{}'.format(lbl)] = answer

using cached model
using cached model


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

Batch Loss:  12.84142864588648 Accuracy:  tensor(0.9766, device='cuda:0')


100%|██████████| 108/108 [00:57<00:00,  1.89it/s]


Train Loss : 13.391136799938977 Accuracy : tensor(0.9774, device='cuda:0')


...

 93%|█████████▎| 100/108 [00:53<00:04,  1.89it/s]

Batch Loss:  0.5607589335995726 Accuracy:  tensor(0.9991, device='cuda:0')


100%|██████████| 108/108 [00:57<00:00,  1.89it/s]


Train Loss : 0.6852455788175575 Accuracy : tensor(0.9991, device='cuda:0')


100%|██████████| 1038/1038 [00:20<00:00, 49.82it/s]


In [40]:

model_name = "ko-electra"

# Test Data 정답
test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))
  
# 비도덕아님 인 경우 예측 결과 만들기
# 비도덕아님은 다른 케이스가 모두 False 인 경우에 True 이다.
moral = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        moral.append(1)
    else :
        moral.append(0)


# 정답 비교
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print("맞춘 갯수 :", count)

accuracy = count / len(origin_test)
print("Label 예측 accuracy :", accuracy)      

# 결과값 저장
result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : moral, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}

result = pd.DataFrame(result)
result.to_csv("result/final_v3_{}.csv".format(model_name))

pred = []
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    pred.append(data)
f1 = f1_score(test_label, pred, average='micro')



test_result = pd.DataFrame({"model_name" : [model_name],
                                    "accuracy" : [accuracy],
                                    "f1_score" : [f1]})
model_result = model_result.append(test_result, ignore_index=True)


맞춘 갯수 : 732
Label 예측 accuracy : 0.7052023121387283


In [41]:
model_result

,model_name,accuracy,f1_score
0,electra-kor-base,0.713873,0.775798
1,kobert,0.660886,0.738848
2,ko-electra,0.705202,0.779730


### ko-electra-v1

In [42]:
# koelectra v1

class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]
    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
    # print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    return input_ids, attention_mask

In [43]:
# koelectra v1

device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

label = ['범죄', '혐오', '선정', '폭력', '비난', '욕설', '차별']

# 모델 학습 파라미터
epochs = 10
batch_size = 128
warmup_ratio=0.1


for lbl in label : 

    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic':labels_t.reshape(-1)})



    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)
    # 모델 학습 파라미터

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t_total = len(train_loader) * epochs
    losses = []
    accuracy = []
    model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-discriminator', num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)
    for i in range(epochs):
      total_loss = 0.0
      correct = 0
      total = 0
      batches = 0

      model.train()
      for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_batch = y_batch.long()
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
          print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
          
      losses.append(total_loss)
      accuracy.append(correct.float() / total)
      print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    model.eval()
    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      answer.append(predicted.item())
      globals()['{}'.format(lbl)] = answer    


using cached model
using cached model


Some weights of the model checkpoint at monologg/koelectra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['clas

Batch Loss:  11.589954318478703 Accuracy:  tensor(0.9646, device='cuda:0')


100%|██████████| 108/108 [00:56<00:00,  1.90it/s]


Train Loss : 12.377157749608159 Accuracy : tensor(0.9653, device='cuda:0')


...

 93%|█████████▎| 100/108 [00:52<00:04,  1.89it/s]

Batch Loss:  6.147740165702999 Accuracy:  tensor(0.9832, device='cuda:0')


100%|██████████| 108/108 [00:56<00:00,  1.90it/s]


Train Loss : 6.4614652413874865 Accuracy : tensor(0.9837, device='cuda:0')


100%|██████████| 1038/1038 [00:20<00:00, 50.77it/s]


In [44]:

model_name = "ko-electra-v1"

# Test Data 정답
test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))
  
# 비도덕아님 인 경우 예측 결과 만들기
# 비도덕아님은 다른 케이스가 모두 False 인 경우에 True 이다.
moral = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        moral.append(1)
    else :
        moral.append(0)


# 정답 비교
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print("맞춘 갯수 :", count)

accuracy = count / len(origin_test)
print("Label 예측 accuracy :", accuracy)      

# 결과값 저장
result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : moral, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}

result = pd.DataFrame(result)
result.to_csv("result/final_v3_{}.csv".format(model_name))

from sklearn.metrics import f1_score
pred = []
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    pred.append(data)
f1 = f1_score(test_label, pred, average='micro')



test_result = pd.DataFrame({"model_name" : [model_name],
                                    "accuracy" : [accuracy],
                                    "f1_score" : [f1]})
model_result = model_result.append(test_result, ignore_index=True)


맞춘 갯수 : 697
Label 예측 accuracy : 0.6714836223506744


In [45]:
model_result

,model_name,accuracy,f1_score
0,electra-kor-base,0.713873,0.775798
1,kobert,0.660886,0.738848
2,ko-electra,0.705202,0.779730
3,ko-electra-v1,0.671484,0.747307


### ko-electra-v2

In [46]:
# KoElectra_v2

class KLUEDataset_train(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator")


  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title', 'topic_idx']].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y
class KLUEDataset_test(Dataset):
  def __init__(self, DataFrame):
    self.dataset = DataFrame
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator")

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.loc[idx, ['title']].values
    text = row[0]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
    )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]
    return input_ids, attention_mask

In [47]:
# KoElectra_v2

label = ['범죄', '혐오', '선정', '폭력', '비난', '욕설', '차별']


device = torch.device("cuda")
bertmodel, vocab = get_pytorch_kobert_model()

# 모델 학습 파라미터
epochs = 10
batch_size = 128
warmup_ratio=0.1


for lbl in label : 


    train = origin_train.copy()
    test = origin_test.copy()
    # 모델 조건에 맞추고, lbl 에 맞추어서 필터링
    titles_t = train.sentence.to_numpy().reshape(-1,1)
    labels_t = train[lbl].to_numpy().reshape(-1,1)
    train = pd.DataFrame({'title':titles_t.reshape(-1), 'topic_idx':labels_t.reshape(-1)})

    titles_t = test.sentence.to_numpy().reshape(-1,1)
    labels_t = test[lbl].to_numpy().reshape(-1,1)
    df_valid = pd.DataFrame({'title':titles_t.reshape(-1), 'topic':labels_t.reshape(-1)})



    train_dataset = KLUEDataset_train(train)
    test_dataset = KLUEDataset_test(df_valid)
    # 모델 학습 파라미터

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t_total = len(train_loader) * epochs
    losses = []
    accuracy = []
    model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v2-discriminator', num_labels=2).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)
    for i in range(epochs):
      total_loss = 0.0
      correct = 0
      total = 0
      batches = 0

      model.train()
      for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_batch = y_batch.long()        
        y_pred = model(input_ids_batch.to(device), attention_mask = attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
          print("Batch Loss: ", total_loss, "Accuracy: ", correct.float() / total)
          
      losses.append(total_loss)
      accuracy.append(correct.float() / total)
      print("Train Loss :", total_loss, "Accuracy :", correct.float() / total)

    answer = []
    model.eval()
    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      answer.append(predicted.item())
      globals()['{}'.format(lbl)] = answer


using cached model
using cached model


Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: 

Batch Loss:  11.944384719245136 Accuracy:  tensor(0.9758, device='cuda:0')


100%|██████████| 108/108 [00:56<00:00,  1.90it/s]


Train Loss : 12.579832718707621 Accuracy : tensor(0.9762, device='cuda:0')


...

 93%|█████████▎| 100/108 [00:53<00:04,  1.89it/s]

Batch Loss:  8.337379679083824 Accuracy:  tensor(0.9837, device='cuda:0')


100%|██████████| 108/108 [00:57<00:00,  1.89it/s]


Train Loss : 9.030447121709585 Accuracy : tensor(0.9837, device='cuda:0')


100%|██████████| 1038/1038 [00:21<00:00, 49.40it/s]


In [50]:

model_name = "ko-electra-v2"

# Test Data 정답
test_label = []
for idx, row in origin_test.iloc[:, [1,2,3,4,5,6,7,8]].iterrows() :
  test_label.append(list(row))
  
# 비도덕아님 인 경우 예측 결과 만들기
# 비도덕아님은 다른 케이스가 모두 False 인 경우에 True 이다.
moral = []
for i, data in enumerate(zip(범죄, 혐오, 선정, 폭력, 비난, 욕설, 차별)) :
    if sum(list(data)) == 0  :
        moral.append(1)
    else :
        moral.append(0)


# 정답 비교
count = 0
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    if list(data) == test_label[i] :
      count +=1
print("맞춘 갯수 :", count)

accuracy = count / len(origin_test)
print("Label 예측 accuracy :", accuracy)      

# 결과값 저장
result = {
    "범죄" : 범죄, 
    "혐오" : 혐오, 
    "선정" : 선정, 
    "비도덕아님" : moral, 
    "폭력" : 폭력, 
    "비난" : 비난, 
    "욕설" : 욕설, 
    "차별" : 차별
}

result = pd.DataFrame(result)
result.to_csv("result/final_v3_{}.csv".format(model_name))

pred = []
for i, data in enumerate(zip(범죄, 혐오, 선정, moral, 폭력, 비난, 욕설, 차별)) :
    pred.append(data)
f1_score = f1_score(test_label, pred, average='micro')



test_result = pd.DataFrame({"model_name" : [model_name],
                                    "accuracy" : [accuracy],
                                    "f1_score" : [f1_score]})
model_result = model_result.append(test_result, ignore_index=True)

model_result.to_csv("result/model_result_v3.csv")

맞춘 갯수 : 704
Label 예측 accuracy : 0.6782273603082851


TypeError: 'numpy.float64' object is not callable

In [49]:
model_result

,model_name,accuracy,f1_score
0,electra-kor-base,0.713873,0.775798
1,kobert,0.660886,0.738848
2,ko-electra,0.705202,0.779730
3,ko-electra-v1,0.671484,0.747307
4,ko-electra-v2,0.678227,0.748344
